In [1]:
import os
import numpy as np
np.random.seed(1969)
import tensorflow as tf
tf.set_random_seed(1969)


from scipy import signal
import glob
import re
import pandas as pd
import gc
from scipy.io import wavfile

# from keras import optimizers, losses, activations, models
# from keras.layers import GRU, Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, Conv3D, ConvLSTM2D
# from keras.callbacks import TensorBoard
# from keras.models import Sequential
from tqdm import tqdm
from sklearn.model_selection import GroupKFold
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
urban = pd.read_csv("UrbanSound8K/metadata/UrbanSound8K.csv")

In [3]:
classes = urban["class"]

In [4]:
L = 16000

In [5]:
def list_wavs_fname(dirpath, ext='wav'):
    fpaths = matches
    pat = r'.+/(\w+)/\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+/(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

In [6]:
def pad_audio(samples):
    if len(samples) >= L: return samples
    else: return np.pad(samples, pad_width=(L - len(samples), 0), mode='constant', constant_values=(0, 0))

def chop_audio(samples, L=16000, num=1000):
    for i in range(num):
        beg = np.random.randint(0, len(samples) - L)
        yield samples[beg: beg + L]

In [7]:
root_path = r'UrbanSound8K'
out_path = r'UrbanSound8K'
model_path = r'.'
train_data_path = os.path.join(r"UrbanSound8K", 'audio')

In [8]:
import fnmatch

In [9]:
matches = []
for root, dirnames, filenames in os.walk(train_data_path):
    for filename in fnmatch.filter(filenames, '*.wav'):
        matches.append(os.path.join(root, filename))

In [10]:
def read(file, readmarkers=False, readmarkerlabels=False, readmarkerslist=False, readloops=False, readpitch=False, normalized=False, forcestereo=False):
    """
    Return the sample rate (in samples/sec) and data from a WAV file
    Parameters
    ----------
    file : file
        Input wav file.
    Returns
    -------
    rate : int
        Sample rate of wav file
    data : numpy array
        Data read from wav file
    Notes
    -----
    * The file can be an open file or a filename.
    * The returned sample rate is a Python integer
    * The data is returned as a numpy array with a
      data-type determined from the file.
    """
    if hasattr(file,'read'):
        fid = file
    else:
        fid = open(file, 'rb')

    fsize = _read_riff_chunk(fid)
    noc = 1
    bits = 8
    #_cue = []
    #_cuelabels = []
    _markersdict = collections.defaultdict(lambda: {'position': -1, 'label': ''})
    loops = []
    pitch = 0.0
    while (fid.tell() < fsize):
        # read the next chunk
        chunk_id = fid.read(4)
        if chunk_id == b'fmt ':
            size, comp, noc, rate, sbytes, ba, bits = _read_fmt_chunk(fid)
        elif chunk_id == b'data':
            data = _read_data_chunk(fid, noc, bits, normalized)
            if data == None:
                return None
        elif chunk_id == b'cue ':
            str1 = fid.read(8)
            size, numcue = struct.unpack('<ii',str1)
            for c in range(numcue):
                str1 = fid.read(24)
                id, position, datachunkid, chunkstart, blockstart, sampleoffset = struct.unpack('<iiiiii', str1)
                #_cue.append(position)
                _markersdict[id]['position'] = position                    # needed to match labels and markers

        elif chunk_id == b'LIST':
            str1 = fid.read(8)
            size, type = struct.unpack('<ii', str1)
        elif chunk_id in [b'ICRD', b'IENG', b'ISFT', b'ISTJ']:    # see http://www.pjb.com.au/midi/sfspec21.html#i5
            _skip_unknown_chunk(fid)
        elif chunk_id == b'labl':
            str1 = fid.read(8)
            size, id = struct.unpack('<ii',str1)
            size = size + (size % 2)                              # the size should be even, see WAV specfication, e.g. 16=>16, 23=>24
            label = fid.read(size-4).rstrip('\x00')               # remove the trailing null characters
            #_cuelabels.append(label)
            _markersdict[id]['label'] = label                           # needed to match labels and markers

        elif chunk_id == b'smpl':
            str1 = fid.read(40)
            size, manuf, prod, sampleperiod, midiunitynote, midipitchfraction, smptefmt, smpteoffs, numsampleloops, samplerdata = struct.unpack('<iiiiiIiiii', str1)
            cents = midipitchfraction * 1./(2**32-1)
            pitch = 440. * 2 ** ((midiunitynote + cents - 69.)/12)
            for i in range(numsampleloops):
                str1 = fid.read(24)
                cuepointid, type, start, end, fraction, playcount = struct.unpack('<iiiiii', str1) 
                loops.append([start, end])
        else:
            warnings.warn("Chunk " + chunk_id + " skipped", WavFileWarning)
            _skip_unknown_chunk(fid)
    fid.close()

    if data.ndim == 1 and forcestereo:
        data = numpy.column_stack((data, data))

    _markerslist = sorted([_markersdict[l] for l in _markersdict], key=lambda k: k['position'])  # sort by position
    _cue = [m['position'] for m in _markerslist]
    _cuelabels = [m['label'] for m in _markerslist]
    
    return (rate, data, bits, ) \
        + ((_cue,) if readmarkers else ()) \
        + ((_cuelabels,) if readmarkerlabels else ()) \
        + ((_markerslist,) if readmarkerslist else ()) \
        + ((loops,) if readloops else ()) \
        + ((pitch,) if readpitch else ())

In [11]:
from __future__ import division, print_function, absolute_import

import numpy
import struct
import warnings
import collections
from operator import itemgetter

class WavFileWarning(UserWarning):
    pass
    
_ieee = False
    
# assumes file pointer is immediately
#  after the 'fmt ' id
def _read_fmt_chunk(fid):
    res = struct.unpack('<ihHIIHH',fid.read(20))
    size, comp, noc, rate, sbytes, ba, bits = res
    if (comp != 1 or size > 16):
        if (comp == 3):
            global _ieee
            _ieee = True
          #warnings.warn("IEEE format not supported", WavFileWarning)        
        else: 
            warnings.warn("Unfamiliar format bytes", WavFileWarning)
        if (size>16):
            fid.read(size-16)
    return size, comp, noc, rate, sbytes, ba, bits

# assumes file pointer is immediately
#   after the 'data' id
def _read_data_chunk(fid, noc, bits, normalized=False):
    size = struct.unpack('<i',fid.read(4))[0]

    if bits == 8 or bits == 24:
        dtype = 'u1'
        bytes = 1
    else:
        bytes = bits//8
        dtype = '<i%d' % bytes
        
    if bits == 32 and _ieee:
        dtype = 'float32'
    if bytes == 0:
        return None
    data = numpy.fromfile(fid, dtype=dtype, count=size//bytes)
    
    if bits == 24:
        a = numpy.empty((len(data) // 3, 4), dtype='u1')
        a[:, :3] = data.reshape((-1, 3))
        a[:, 3:] = (a[:, 3 - 1:3] >> 7) * 255
        data = a.view('<i4').reshape(a.shape[:-1])
    
    if noc > 1:
        data = data.reshape(-1,noc)
        
    if bool(size & 1):     # if odd number of bytes, move 1 byte further (data chunk is word-aligned)
        fid.seek(1,1)    

    if normalized:
        if bits == 8 or bits == 16 or bits == 24: 
            normfactor = 2 ** (bits-1)
        data = numpy.float32(data) * 1.0 / normfactor

    return data

def _skip_unknown_chunk(fid):
    data = fid.read(4)
    size = struct.unpack('<i', data)[0]
    if bool(size & 1):     # if odd number of bytes, move 1 byte further (data chunk is word-aligned)
        size += 1 
    fid.seek(size, 1)

def _read_riff_chunk(fid):
    str1 = fid.read(4)
    if str1 != b'RIFF':
        raise ValueError("Not a WAV file.")
    fsize = struct.unpack('<I', fid.read(4))[0] + 8
    str2 = fid.read(4)
    if (str2 != b'WAVE'):
        raise ValueError("Not a WAV file.")
    return fsize

In [12]:
read(matches[0])[1]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:25: WavFileWarning: Unfamiliar format bytes
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:75: WavFileWarning: Chunk fact skipped
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


array([[ -318927, -1755856],
       [-1835623, -1538056],
       [-1401212, -1540951],
       ..., 
       [ 1342852,   442350],
       [  845104,  -962313],
       [    1221, -1216181]], dtype=int32)

In [13]:
matches[0]

'UrbanSound8K/audio/fold6/166931-4-2-13.wav'

In [90]:
new_sample_rate=16000
y_train = []
x_train = np.zeros((8732,199,26),np.float32)
G = []
ix = 0
for i in tqdm(range(matches)):
    match = matches[i]
    s_ = read(match)
    if s_ == None:
        continue
    sample_rate = s_[0]
    samples = s_[1]
    samples = pad_audio(samples)
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else:
        n_samples = [samples]
    ok = 0
    for samples in n_samples:
        filter_banks = logfbank(samples)
        filter_banks -= (np.mean(filter_banks, axis=0) + 1e-8)
#         print(filter_banks.shape)
        if filter_banks.shape != (199,26):
            continue
        ok = 1
        x_train[ix,:,:] = filter_banks
        break
    if ok == 0:
        continue
    m = re.search('/([^/]+)$', match)
    if m:
        found = m.group(0)
    y_train.append(list(urban.loc[urban['slice_file_name'] == found[1:]]["class"])[0])
    ix += 1

100%|██████████| 8732/8732 [47:41<00:00,  3.05it/s] 


In [91]:
set(y_train)

{'air_conditioner',
 'car_horn',
 'children_playing',
 'dog_bark',
 'drilling',
 'engine_idling',
 'gun_shot',
 'jackhammer',
 'siren',
 'street_music'}

In [92]:
x_train_size = list(x_train.shape)
x_train_size.append(1)
x_train = x_train.reshape(x_train_size)
x_train = np.array(x_train)
y_train = np.array(y_train)
def label_transform(labels):
    return pd.get_dummies(pd.Series(labels))
y_train = label_transform(y_train)
y_train = np.array(y_train)
x_train = x_train[:len(y_train)]

index = range(len(x_train))
import random
random.shuffle(index)

In [100]:
num_train = 7000
train = index[:num_train]
val = index[num_train:]

In [154]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Concatenate
from keras.layers import concatenate
from keras.models import Model
from keras.layers import Input, merge, ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import keras.backend as K

from custom_layers import Scale

def DenseNet(input_shape=(99, 26, 1),nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.0, dropout_rate=0.0, weight_decay=1e-4, classes=12, weights_path=None):
    '''Instantiate the DenseNet architecture,
        # Arguments
            nb_dense_block: number of dense blocks to add to end
            growth_rate: number of filters to add per dense block
            nb_filter: initial number of filters
            reduction: reduction factor of transition blocks.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            classes: optional number of classes to classify images
            weights_path: path to pre-trained weights
        # Returns
            A Keras model instance.
    '''
    eps = 1.1e-5

    # compute compression factor
    compression = 1.0 - reduction

    # Handle Dimension Ordering for different backends
    global concat_axis
    concat_axis = 3
    img_input = Input(shape=input_shape, name='data')

    # From architecture for ImageNet (Table 1 in the paper)
    nb_filter = 64
    nb_layers = [6,12,32,32] # For DenseNet-169
    
    

    # Initial convolution
    x = ZeroPadding2D((10, 10), name='conv1_zeropadding')(img_input)
    x1 = Conv2D(1, 3,dilation_rate=1,padding="same")(x)
    x2 = Conv2D(1, 3,dilation_rate=2,padding="same")(x)
    x3 = Conv2D(1, 3,dilation_rate=3,padding="same")(x)
    x4 = Conv2D(1, 3,dilation_rate=4,padding="same")(x)
    x = concatenate([x1,x2,x3,x4])
    
    x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=concat_axis, name='conv1_bn')(x)
    x = Scale(axis=concat_axis, name='conv1_scale')(x)
    x = Activation('relu', name='relu1')(x)
    x = ZeroPadding2D((1, 1), name='pool1_zeropadding')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx+2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)

        # Add transition_block
        x = transition_block(x, stage, nb_filter, compression=compression, dropout_rate=dropout_rate, weight_decay=weight_decay)
        nb_filter = int(nb_filter * compression)

    final_stage = stage + 1
    x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)

    x = BatchNormalization(epsilon=eps, axis=concat_axis, name='conv'+str(final_stage)+'_blk_bn')(x)
    x = Scale(axis=concat_axis, name='conv'+str(final_stage)+'_blk_scale')(x)
    x = Activation('relu', name='relu'+str(final_stage)+'_blk')(x)
    x = GlobalAveragePooling2D(name='pool'+str(final_stage))(x)

    x = Dense(classes, name='fc6')(x)
    x = Activation('softmax', name='prob')(x)

    model = Model(img_input, x, name='densenet')

    if weights_path is not None:
        model.load_weights(weights_path)

    return model


def conv_block(x, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
    '''Apply BatchNorm, Relu, bottleneck 1x1 Conv2D, 3x3 Conv2D, and option dropout
        # Arguments
            x: input tensor 
            stage: index for dense block
            branch: layer index within each dense block
            nb_filter: number of filters
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''
    eps = 1.1e-5
    conv_name_base = 'conv' + str(stage) + '_' + str(branch)
    relu_name_base = 'relu' + str(stage) + '_' + str(branch)

    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4  
    x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base+'_x1_bn')(x)
    x = Scale(axis=concat_axis, name=conv_name_base+'_x1_scale')(x)
    x = Activation('relu', name=relu_name_base+'_x1')(x)
    x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 Convolution
    x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base+'_x2_bn')(x)
    x = Scale(axis=concat_axis, name=conv_name_base+'_x2_scale')(x)
    x = Activation('relu', name=relu_name_base+'_x2')(x)
    x = ZeroPadding2D((1, 1), name=conv_name_base+'_x2_zeropadding')(x)
    x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x


def transition_block(x, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
    ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout 
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_filter: number of filters
            compression: calculated as 1 - reduction. Reduces the number of feature maps in the transition block.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''

    eps = 1.1e-5
    conv_name_base = 'conv' + str(stage) + '_blk'
    relu_name_base = 'relu' + str(stage) + '_blk'
    pool_name_base = 'pool' + str(stage) 

    x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base+'_bn')(x)
    x = Scale(axis=concat_axis, name=conv_name_base+'_scale')(x)
    x = Activation('relu', name=relu_name_base)(x)
    x = Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling2D((2, 2), strides=(2, 2), name=pool_name_base)(x)

    return x


def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''

    eps = 1.1e-5
    concat_feat = x

    for i in range(nb_layers):
        branch = i+1
        x = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate, weight_decay)
        concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))

        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [172]:
model1 = DenseNet(input_shape=(199, 26, 1),classes=10)

In [21]:
from keras import optimizers

In [149]:
from keras.utils import multi_gpu_model
parallel_model = multi_gpu_model(model1, gpus=4)

In [156]:
sgd = optimizers.SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True)

parallel_model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['categorical_accuracy'])

In [30]:
NUM_FOLDS = 4
EPOCHS = 30
BATCH_SIZE = 128
BAGS = 16

kf = GroupKFold(n_splits=NUM_FOLDS)

shape = None

In [ ]:
history = parallel_model.fit(x_train[train], y_train[train], batch_size=BATCH_SIZE, validation_data=(x_train[val], y_train[val]), epochs=EPOCHS, shuffle=True, verbose=1, callbacks=[])

In [96]:
history = parallel_model.fit(x_train[train], y_train[train], batch_size=BATCH_SIZE, validation_data=(x_train[val], y_train[val]), epochs=EPOCHS, shuffle=True, verbose=1, callbacks=[])

In [102]:
history = parallel_model.fit(x_train[train], y_train[train], batch_size=BATCH_SIZE, epochs=1, shuffle=True, verbose=1, callbacks=[])

Epoch 1/1
7000/7000 [==============================] - 28s 4ms/step - loss: 0.0189 - categorical_accuracy: 0.9827


In [118]:
L = 16000
legal_labels = 'yes no up down left right on off stop go silence unknown'.split()

root_path = r'.'
out_path = r'.'
model_path = r'.'
train_data_path = os.path.join(r".", 'input', 'train', 'audio')
test_data_path = os.path.join(r".", 'input', 'test', 'audio')

In [115]:
def label_transform(labels):
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('silence')
        elif label not in legal_labels:
            nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))

In [121]:
def list_wavs_fname(dirpath, ext='wav'):
    print(dirpath)
    fpaths = glob.glob(os.path.join(dirpath, r'*/*' + ext))
    pat = r'.+/(\w+)/\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+/(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

In [122]:
labels, fnames = list_wavs_fname(train_data_path)
new_sample_rate=16000
y_train = []
x_train = np.zeros((64727,99,26),np.float32)
G = []
ix = 0
for label, fname in zip(labels, fnames):
    sample_rate, samples = wavfile.read(os.path.join(train_data_path, label, fname))
    samples = pad_audio(samples)
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else:
        n_samples = [samples]
    for samples in n_samples:
        filter_banks = logfbank(samples)
        filter_banks -= (np.mean(filter_banks, axis=0) + 1e-8)
        x_train[ix,:,:] = filter_banks
    y_train.append(label)
    group = fname.split('_')[0]
    G.append(group)
    ix += 1

./input/train/audio


In [123]:
x_train_shape = x_train.shape
x_train_shape = list(x_train_shape)
x_train_shape.append(1)
x_train = x_train.reshape(x_train_shape)
x_train.shape

(64727, 99, 26, 1)

In [124]:
y_train = np.array(y_train)
y_train = label_transform(y_train)
y_train = np.array(y_train)

In [125]:
index = range(len(x_train))
random.shuffle(index)
num_train = 58000
train = index[:num_train]
val = index[num_train:]

In [126]:
NUM_FOLDS = 4
EPOCHS = 30
BATCH_SIZE = 256
BAGS = 16

kf = GroupKFold(n_splits=NUM_FOLDS)

shape = None

In [137]:
new_model = DenseNet(input_shape=(99, 26, 1),classes=12)

In [143]:
len(new_model.layers)

854

In [173]:
new_model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['categorical_accuracy'])

In [175]:
for i in tqdm(range(1,len(new_model.layers))):
    if len(model1.layers[i].get_weights()):
        new_model.layers[i].set_weights(model1.layers[i].get_weights())

100%|█████████▉| 849/853 [1:25:26<00:24,  6.04s/it]


ValueError: Layer weight shape (2688, 12) not compatible with provided weight shape (2688, 10)

In [176]:
history = model1.fit(x_train[train], y_train[train], batch_size=BATCH_SIZE, validation_data=(x_train[val], y_train[val]), epochs=EPOCHS, shuffle=True, verbose=1, callbacks=[])

Train on 58000 samples, validate on 6727 samples
Epoch 1/30
58000/58000 [==============================] - 73s 1ms/step - loss: 0.1728 - categorical_accuracy: 0.6319 - val_loss: 0.1542 - val_categorical_accuracy: 0.6499
Epoch 2/30
58000/58000 [==============================] - 71s 1ms/step - loss: 0.1376 - categorical_accuracy: 0.6763 - val_loss: 0.1236 - val_categorical_accuracy: 0.7012
Epoch 3/30
58000/58000 [==============================] - 70s 1ms/step - loss: 0.1058 - categorical_accuracy: 0.7618 - val_loss: 0.0929 - val_categorical_accuracy: 0.7977
Epoch 4/30
58000/58000 [==============================] - 70s 1ms/step - loss: 0.0799 - categorical_accuracy: 0.8272 - val_loss: 0.0808 - val_categorical_accuracy: 0.8134
Epoch 5/30
58000/58000 [==============================] - 70s 1ms/step - loss: 0.0634 - categorical_accuracy: 0.8643 - val_loss: 0.0659 - val_categorical_accuracy: 0.8530
Epoch 6/30
58000/58000 [==============================] - 70s 1ms/step - loss: 0.0533 - categori

KeyboardInterrupt: 

In [ ]:
y_train = label_transform(labels)
label_index = y_train.columns.values
y_train = y_train.values
y_train = np.array(y_train)

In [ ]:
def test_data_generator(batch=32):
    fpaths = glob.glob(os.path.join(test_data_path, '*wav'))
    i = 0
    for path in fpaths:
        if i == 0:
            imgs = []
            fnames = []
        i += 1
        rate, samples = wavfile.read(path)
        samples = pad_audio(samples)
        filter_banks = logfbank(samples)
        filter_banks -= (np.mean(filter_banks, axis=0) + 1e-8)
        imgs.append(filter_banks)
        fnames.append(path.split('/')[-1])
        if i == batch:
            i = 0
            imgs = np.array(imgs)
            yield fnames, imgs
    if i < batch:
        imgs = np.array(imgs)

        yield fnames, imgs
    raise StopIteration()



gc.collect()

index = []
results = []
probs = []
for fnames, imgs in tqdm(test_data_generator(batch=32)):
    li = list(imgs.shape)
    li.append(1)
    imgs = imgs.reshape(tuple(li))
    predicts = model1.predict(imgs)
    probs.extend(predicts)
    predicts = np.argmax(predicts, axis=1)
    predicts = [label_index[p] for p in predicts]
    index.extend(fnames)
    results.extend(predicts)

df = pd.DataFrame(columns=['fname', 'label'])
df['fname'] = index
df['label'] = results


In [181]:
df.to_csv(os.path.join(out_path, 'gru_sub_final_.csv'), index=False)